In [ ]:
import os
import shutil

import mne
import nibabel as nib
import numpy as np
from lameg.invert import coregister, invert_ebb, load_source_time_series
from lameg.laminar import sliding_window_model_comparison
from lameg.util import get_fiducial_coords, get_surface_names, load_meg_sensor_data, get_bigbrain_layer_boundaries
import spm_standalone
import matlab
import h5py
from mne import read_epochs
from scipy.spatial import cKDTree, KDTree
from scipy.stats import zscore
import matplotlib.pyplot as plt
import pandas as pd
import glob

def get_fiducial_coords(SUB, file_path):
    """
    Get the fidicials from each participants in the specified file 
    coordinates are reconstructed and with FS RAS offset
    """

    for file in os.listdir(file_path):
        if file.endswith(".tsv"):
            full_file_path = os.path.join(file_path,file)
            df = pd.read_csv(full_file_path, sep='\t')
            print("Found tsv file for fiducials")

            data = df[df["Subject ID"] == SUB]
            nas = data.iloc[:, 5].values
            nas = [float(item) for item in nas[0].split(', ')]

            lpa = data.iloc[:, 6].values 
            lpa = [float(item) for item in lpa[0].split(', ')]

            rpa = data.iloc[:, 7].values
            rpa = [float(item) for item in rpa[0].split(', ')]

            return nas, lpa, rpa
        
def upload_meg_file(rawdata_path, sub, ses, condition):
    """
    Search and upload the raw datachunck

    :param rawdata_path: datapath
    :param sub: subjects id
    :param ses: session id 
    :param condition: experimental condition (block-wise)
    """
    
    sub_meg_path = os.path.join(rawdata_path, "sub-" + sub , "ses-0" + ses , "meg/sub-"+ sub + "_ses-0" + ses + "_task-" + condition + "_acq-")
    folders_acq = glob.glob(sub_meg_path + "*_meg.ds", recursive=True)
    list_folders = sorted(folders_acq, key=lambda x: int(x.split('acq-')[1].split('_')[0]))
    
    return list_folders

def get_roi_idx(subj_id, surf_dir, hemi, regions, surf):
    """
    Returns the vertex indices from a downsampled surface mesh that correspond
    to specified anatomical regions (from FreeSurfer annotations).

    Parameters:
    - subj_id (str): FreeSurfer subject ID.
    - surf_dir (str): Path to directory containing the postprocessed surfaces (e.g., pial.gii).
    - hemi (str or None): Hemisphere ('lh', 'rh', or None for both).
    - regions (list of str): List of region names to extract from the annotation.
    - surf (nib.GiftiImage): Downsampled surface mesh used for laminar inference.

    Returns:
    - roi_idx (np.ndarray): Array of indices into `surf` corresponding to the selected ROI.
    """
    fs_subjects_dir = os.getenv('SUBJECTS_DIR')
    fs_subject_dir = os.path.join(fs_subjects_dir, subj_id)

    roi_idx = []
    hemis = []
    if hemi is None:
        hemis.extend(['lh', 'rh'])
    else:
        hemis.append(hemi)
    for hemi in hemis:
        pial = nib.load(os.path.join(surf_dir, f'{hemi}.pial.gii'))

        #         annotation = os.path.join(fs_subject_dir, 'label', f'{hemi}.aparc.annot')
        annotation = os.path.join(fs_subject_dir, 'label', f'{hemi}.BA_exvivo.annot')
        label, ctab, names = nib.freesurfer.read_annot(annotation)

        name_indices = [names.index(region.encode()) for region in regions]
        orig_vts = np.where(np.isin(label, name_indices))[0]

        # Find the original vertices closest to the downsampled vertices
        kdtree = KDTree(pial.darrays[0].data[orig_vts, :])
        # Calculate the percentage of vertices retained
        dist, vert_idx = kdtree.query(surf.darrays[0].data, k=1)
        hemi_roi_idx = np.where(dist == 0)[0]
        roi_idx = np.union1d(roi_idx, hemi_roi_idx)
    return roi_idx.astype(int)


def get_cortical_thickness(multilayer_mesh, n_layers):
    """
    Estimates cortical thickness at each vertex from a multilayer surface mesh.

    Parameters:
    - multilayer_mesh (nib.GiftiImage): Combined mesh of all depth layers (equidistant).
    - n_layers (int): Total number of cortical depth surfaces in the mesh.

    Returns:
    - thickness (np.ndarray): Vertex-wise Euclidean distance between layer 1 and layer N.
    """
    verts_per_surf = int(multilayer_mesh.darrays[0].data.shape[0] / n_layers)
    thickness = np.sqrt(np.sum((multilayer_mesh.darrays[0].data[:verts_per_surf, :] - multilayer_mesh.darrays[0].data[
                                                                                      (n_layers - 1) * verts_per_surf:,
                                                                                      :]) ** 2, axis=-1))
    return thickness


def get_lead_field_rmse(gainmat_fname, n_layers, verts_per_surf):
    """
    Computes the RMSE of lead field vectors across cortical depths
    relative to the superficial (layer 1) model, for each vertex.

    Parameters:
    - gainmat_fname (str): Path to HDF5 file containing the gain matrix ('G').
    - n_layers (int): Number of cortical depth layers.
    - verts_per_surf (int): Number of vertices per layer.

    Returns:
    - rmse (np.ndarray): RMSE between deep and superficial lead fields per vertex.
    """
    with h5py.File(gainmat_fname, 'r') as file:
        lf_mat = file['G'][()]

    layer_lf_mat = np.zeros((verts_per_surf, n_layers, lf_mat.shape[1]))
    diff_layer_lf_mat = np.zeros((verts_per_surf, n_layers))
    for i in range(n_layers):
        layer_lf_mat[:, i, :] = lf_mat[i * verts_per_surf:(i + 1) * verts_per_surf, :]
    for j in range(verts_per_surf):
        for i in range(n_layers):
            diff_layer_lf_mat[j, i] = np.sqrt(np.mean((layer_lf_mat[j, i, :] - layer_lf_mat[j, 0, :]) ** 2))
    return diff_layer_lf_mat[:, -1]


def get_orientation(scalp_mesh_fname, multilayer_mesh, pial_ds_mesh, verts_per_surf):
    """
    Computes the alignment between cortical column orientation vectors and
    local scalp surface normals for each vertex.

    Parameters:
    - scalp_mesh_fname (str): Path to scalp surface (GIFTI).
    - multilayer_mesh (nib.GiftiImage): Mesh with orientation vectors per vertex.
    - pial_ds_mesh (nib.GiftiImage): Downsampled pial mesh for mapping.
    - verts_per_surf (int): Number of vertices per cortical surface.

    Returns:
    - dot_products (np.ndarray): Cosine similarity (abs dot product) between
                                 dipole and scalp normals per vertex.

    Notes:
    - Higher values indicate more radial orientations
    """
    scalp_mesh = nib.load(scalp_mesh_fname)
    scalp_vertices = scalp_mesh.darrays[1].data  # Vertex coordinates
    scalp_faces = scalp_mesh.darrays[0].data  # Face indices

    pial_ds_vertices = pial_ds_mesh.darrays[0].data  # Vertex coordinates
    multilayer_orientations = multilayer_mesh.darrays[2].data  # Orientation vectors

    # Compute surface normals
    def compute_normals(vertices, faces):
        normals = np.zeros_like(vertices)
        for i in range(faces.shape[0]):
            v0, v1, v2 = vertices[faces[i]]
            # Edge vectors
            edge1 = v1 - v0
            edge2 = v2 - v0
            # Cross product to get normal
            normal = np.cross(edge1, edge2)
            normal /= np.linalg.norm(normal)  # Normalize the normal
            normals[faces[i]] += normal
        normals /= np.linalg.norm(normals, axis=1)[:, np.newaxis]  # Normalize all normals
        return normals

    normals = compute_normals(scalp_vertices, scalp_faces)

    # Build a KD-tree for the scalp vertices
    tree = cKDTree(scalp_vertices)

    # Find the nearest neighbor in the scalp mesh for each vertex in the downsampled cortical mesh
    _, indices = tree.query(pial_ds_vertices)

    # Compute the dot product between the orientation and the normal vectors
    dot_products = np.abs(np.einsum('ij,ij->i', normals[indices], multilayer_orientations[:verts_per_surf, :]))
    return dot_products


def get_dist_to_scalp(scalp_mesh_fname, pial_ds_mesh):
    """
    Computes the Euclidean distance from each vertex on the cortical surface
    to the closest point on the scalp.

    Parameters:
    - scalp_mesh_fname (str): Path to scalp surface (GIFTI).
    - pial_ds_mesh (nib.GiftiImage): Downsampled pial surface.

    Returns:
    - distances (np.ndarray): Distance (in mm) from cortex to nearest scalp point per vertex.
    """
    scalp_mesh = nib.load(scalp_mesh_fname)
    scalp_vertices = scalp_mesh.darrays[1].data  # Vertex coordinates

    pial_ds_vertices = pial_ds_mesh.darrays[0].data  # Vertex coordinates

    # Build a KD-tree for the scalp vertices
    tree = cKDTree(scalp_vertices)

    # Find the nearest neighbor in the scalp mesh for each vertex in the downsampled cortical mesh
    distances, _ = tree.query(pial_ds_vertices)

    return distances


def convert_mgz_to_nii(mri_fname, mri_out_path):
    """
    Convert mgz files to .nii - the orig.mgz fpt the mutilayer surface and the T1.mgz for the SPM segmentation
    """
    # function to convert .mgz and to nii file for coregistration

    if mri_fname.endswith(".mgz"):
        nii_fname_path = os.path.join(mri_out_path, os.path.basename(mri_fname)[:-4] + ".nii")
        if not os.path.exists(nii_fname_path):
            print("The .nii file does not exist. Conversion.")
            img = nib.load(mri_fname)
            nib.save(img, nii_fname_path)
            print(f"Conversion finished find .nii file {nii_fname_path}")
        else:
            print(f"File .nii already exists at {nii_fname_path}. Skipping conversion.")
        return nii_fname_path

In [ ]:

meg_rawdata_path = ""
braindyn_files = ""


SUB = ['105', '107', '109', '113', '114',
           '115', '117','118','119', '123', 
           '124', '126','127','129', '130',
           '131','132','133','134','135', 
           '136', '137','138', '141','142',
           '143','144','145']
ses = "1"
condition = "V100V100"
mxf_option = "no_mxf" 
triggers = ["left_stim", "right_stim"] 
n_layers = 11

base_out_dir = os.path.join('')

def run(spm):
    for s, sub in enumerate(SUB): 
        print(f"Now processing part {sub}")
        session = 'ses-01'
        subject = 'sub-' + sub
        out_dir = os.path.join(base_out_dir, subject, session)
        if not os.path.exists(os.path.join(base_out_dir, subject)):
            os.mkdir(os.path.join(base_out_dir, subject))
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)

        mri_data_path = braindyn_files
        mri_out_path = os.path.join(mri_data_path, f"{sub}-synth", "mri")
        mri_fname_orig = os.path.join(mri_data_path, f"{sub}-synth", "mri", "T1.mgz")
        mri_fname = convert_mgz_to_nii(mri_fname_orig, mri_out_path)

        subj_surf_dir = os.path.join(mri_data_path, f'{sub}-synth', 'layer_surf')
        multilayer_mesh_fname = os.path.join(subj_surf_dir, f'sub-{sub}-multilayer.11.ds.link_vector.fixed.gii')
        multilayer_mesh = nib.load(multilayer_mesh_fname)
        verts_per_surf = int(multilayer_mesh.darrays[0].data.shape[0] / n_layers)
        ds_pial = nib.load(os.path.join(subj_surf_dir, 'pial.ds.gii'))
        layer_fnames = get_surface_names(
                n_layers,
                subj_surf_dir,
                'link_vector.fixed')


        nas, lpa, rpa = get_fiducial_coords(sub, braindyn_files)

        print(nas)
        print(lpa)
        print(rpa)

        for trigger in triggers:
            out_fname = os.path.join(base_out_dir, f'results_{subject}_{session}_{trigger}.npz')

            orig_data_file = f"mspm_sub-{sub}_ses-0{ses}_task-{condition}_meg_{mxf_option}_{trigger}_nrg_ERF_epo.mat"
            list_folders = upload_meg_file(meg_rawdata_path, sub, ses, condition)
            mxf_path_dir = os.path.join(list_folders[-1], f"mxf_{mxf_option}_data") 
            data_dir = os.path.join(mxf_path_dir, "mxf_sess_data", "spm_convert_epo")

            base_ERF_fname = os.path.join(data_dir,orig_data_file)
            data_base = os.path.splitext(orig_data_file)[0]

                # Copy data files to tmp directory if we need simulations
            shutil.copy(
                    os.path.join(data_dir, f'{data_base}.mat'),
                    os.path.join(out_dir, f'{data_base}.mat')
                )
            shutil.copy(
                    os.path.join(data_dir, f'{data_base}.dat'),
                    os.path.join(out_dir, f'{data_base}.dat')
                )

            base_fname = os.path.join(out_dir, f'{data_base}.mat')

                # Coregister data to multilayer mesh
            coregister(
                    nas,
                    lpa,
                    rpa,
                    mri_fname,
                    multilayer_mesh_fname,
                    base_fname,
                    spm_instance=spm,
                    viz=False
                )

            patch_size = 5
                # Run localizer on multilayer mesh on 50 to 150ms time window
            [_, _, MU] = invert_ebb(
                    multilayer_mesh_fname,
                    base_fname,
                    n_layers,
                    woi=[50, 150],
                    patch_size=patch_size,
                    n_temp_modes=4,
                    n_spatial_modes=60,
                    # n_spatial_modes=rank['mag'],
                    return_mu_matrix=True,
                    spm_instance=spm,
                    viz=False
                )

            lh_roi_idx = get_roi_idx(f"{sub}-synth", subj_surf_dir, 'lh', ['V1_exvivo'], ds_pial)
            rh_roi_idx = get_roi_idx(f"{sub}-synth", subj_surf_dir, 'rh', ['V1_exvivo'], ds_pial)


            all_layer_ts, ts_time, _ = load_source_time_series(
                    base_fname,
                    mu_matrix=MU,
                    vertices=(5 - 1) * verts_per_surf + np.arange(verts_per_surf)
                )

            base_t_idx = np.where((ts_time >= -200) & (ts_time <= -100))[0]
            m_base = np.mean(np.abs(all_layer_ts[:, base_t_idx]), axis=-1)
            t_idx = np.where((ts_time >= 50) & (ts_time <= 150))[0]
            signal_mag = np.mean(np.abs(all_layer_ts[:, t_idx]), axis=-1) - m_base

                # Compute anatomical predictors
            thickness = get_cortical_thickness(multilayer_mesh, n_layers)
            gainmat_fname = os.path.join(out_dir, f'SPMgainmatrix_{data_base}_1.mat')
            lf_rmse = get_lead_field_rmse(gainmat_fname, n_layers, verts_per_surf)
            scalp_mesh_fname = os.path.join(mri_data_path, f'{sub}-synth', 'mri/T1scalp_2562.surf.gii')
            orientations = get_orientation(scalp_mesh_fname, multilayer_mesh, ds_pial, verts_per_surf)
            distances = get_dist_to_scalp(scalp_mesh_fname, ds_pial)

                # Z-score each anatomical predictor (note: invert distance to scalp)
            z_thickness = zscore(thickness)
            z_lf_rmse = zscore(lf_rmse)
            z_orient = zscore(orientations)
            z_inv_dist = zscore(-distances)

                # Composite anatomical score
            anatomical_score = z_thickness + z_lf_rmse + z_orient + z_inv_dist

            perc_thresh = 99

                # Restrict to roi_idx - LH
            roi_anat_score = anatomical_score[lh_roi_idx]
            roi_signal = signal_mag[lh_roi_idx]

                # Select top 1% signal vertices within ROI
            signal_threshold = np.percentile(roi_signal, perc_thresh)
            high_signal_mask = roi_signal >= signal_threshold
            candidate_indices = lh_roi_idx[high_signal_mask]

                # Among them, select the vertex with the best anatomical suitability
            candidate_anat_scores = anatomical_score[candidate_indices]
            best_idx_local = np.argmax(candidate_anat_scores)
            lh_prior = candidate_indices[best_idx_local]

                # Get source time series from middle surface layer
            lh_prior_ts = all_layer_ts[lh_prior, :]

            # Run sliding time window model comparison
            [lh_Fs, wois] = sliding_window_model_comparison(
                    lh_prior,
                    nas,
                    lpa,
                    rpa,
                    mri_fname,
                    layer_fnames,
                    base_fname,
                    spm_instance=spm,
                    viz=False,
                    invert_kwargs={
                        'patch_size': patch_size,
                        'n_temp_modes': 1,
                        'n_spatial_modes': 60,
                        'win_size': 10,
                        'win_overlap': True
                    }
                )


                # Restrict to roi_idx - RH
            roi_anat_score = anatomical_score[rh_roi_idx]
            roi_signal = signal_mag[rh_roi_idx]

                # Select top 1% signal vertices within ROI
            signal_threshold = np.percentile(roi_signal, perc_thresh)
            high_signal_mask = roi_signal >= signal_threshold
            candidate_indices = rh_roi_idx[high_signal_mask]

                # Among them, select the vertex with the best anatomical suitability
            candidate_anat_scores = anatomical_score[candidate_indices]
            best_idx_local = np.argmax(candidate_anat_scores)
            rh_prior = candidate_indices[best_idx_local]

                # Get source time series from middle surface layer
            rh_prior_ts = all_layer_ts[rh_prior, :]


                # Run sliding time window model comparison
            [rh_Fs, wois] = sliding_window_model_comparison(
                    rh_prior,
                    nas,
                    lpa,
                    rpa,
                    mri_fname,
                    layer_fnames,
                    base_fname,
                    spm_instance=spm,
                    viz=False,
                    invert_kwargs={
                        'patch_size': patch_size,
                        'n_temp_modes': 1,
                        'n_spatial_modes': 60,
                        'win_size': 10,
                        'win_overlap': True
                    }
                )

            np.savez(
                    out_fname,
                    subject=subject,
                    session=session,
                    lh_prior=lh_prior,
                    rh_prior=rh_prior,
                    lh_prior_ts=lh_prior_ts,
                    rh_prior_ts=rh_prior_ts,
                    ts_time=ts_time,
                    lh_Fs=lh_Fs,
                    rh_Fs=rh_Fs,
                    wois=wois
                )
            
            
if __name__ == '__main__':
    spm = spm_standalone.initialize()
    run(spm)
    spm.terminate()        